In [1]:
# !pip install shap
# !pip install PyMuPDF
# !pip show pandas scikit-learn PyMuPDF
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import shap

Name: pandas
Version: 2.3.1
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License

 Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
 All rights reserved.

 Copyright (c) 2011-2023, Open source contributors.

 Redistribution and use in source and binary forms, with or without
 modification, are permitted provided that the following conditions are met:

 * Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

 * Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

 * Neither the name of the copyright holder nor the names of its
   contribut

## 샘플 데이터 만들기 - Feature Engineering

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# --- 1. 데이터 생성을 위한 기본 설정 ---
num_samples = 500
np.random.seed(42) # 재현성을 위해 시드값 고정

# --- 2. 각 특성별 데이터 생성 ---
# 실거래가 정보 (단위: 만 원) - 정규 분포를 따르는 매매가
real_prices = np.random.normal(loc=40000, scale=10000, size=num_samples).astype(int)
real_prices = np.where(real_prices < 10000, 10000, real_prices) # 최소 1억으로 설정

# 건축물 유형 (Categorical)
building_types = np.random.choice(['아파트', '빌라', '오피스텔', '단독주택', '다가구주택'], num_samples, p=[0.4, 0.3, 0.15, 0.1, 0.05])

# 불법 건축 여부 (Boolean)
is_illegal_build = np.random.choice([0, 1], num_samples, p=[0.9, 0.1])

# 건축물 용도 (Categorical)
building_uses = np.random.choice(['주거용', '근린생활시설', '업무시설'], num_samples, p=[0.75, 0.2, 0.05])

# 용도 위반 여부 (Boolean)
is_use_violation = np.where(building_uses != '주거용', np.random.choice([0, 1], num_samples, p=[0.7, 0.3]), 0)

# 근저당권 개수, 소유자변경 횟수 (Count)
num_mortgages = np.random.choice([0, 1, 2, 3], num_samples, p=[0.5, 0.3, 0.15, 0.05])
num_owner_changes = np.random.choice([1, 2, 3, 4], num_samples, p=[0.6, 0.2, 0.1, 0.1])

# 채권최고액 (단위: 만 원) - 근저당권 개수에 따라 다르게 설정
max_claim_amount = np.where(num_mortgages > 0, num_mortgages * np.random.randint(1000, 5000, num_samples), 0)

# 신탁 등기여부, 압류/가압류 등기 여부 (Boolean)
is_trust_registered = np.random.choice([0, 1], num_samples, p=[0.95, 0.05])
is_seizure_registered = np.random.choice([0, 1], num_samples, p=[0.9, 0.1])

# 선순위 채권 존재여부 (Boolean) - 근저당권이나 압류가 있을 경우 존재
has_senior_debt = np.where((num_mortgages > 0) | (is_seizure_registered == 1), 1, 0)

# 전입 가능여부 (Boolean) - 신탁 등기가 있으면 전입 불가능한 경우가 많음
is_movein_possible = np.where(is_trust_registered == 1, 0, np.random.choice([0, 1], num_samples, p=[0.1, 0.9]))

# 우선변제권 여부 (Boolean) - 전입 가능여부가 True이고 다른 선순위 채권이 적을 때 높은 확률
has_priority_right = np.where((is_movein_possible == 1) & (has_senior_debt == 0), 1, 0)

# 등기부 갑구/을구 횟수
num_gapgu = num_owner_changes + np.random.randint(0, 3, num_samples)
num_eulgu = num_mortgages + np.random.randint(0, 2, num_samples)

# 전세가율 (%) - 위험 여부를 판단하는 핵심 특성
jeonse_prices = real_prices * np.random.uniform(low=0.5, high=1.0, size=num_samples)
jeonse_prices = jeonse_prices.astype(int)
jeonse_ratio = (jeonse_prices / real_prices) * 100

# 근저당권 설정일 중 최근날짜 (Random Date)
recent_mortgage_date = pd.to_datetime('2024-01-01') + pd.to_timedelta(np.random.randint(1, 365, num_samples), unit='D')


# --- 3. 생성된 특성들을 DataFrame으로 합치기 ---
df = pd.DataFrame({
    '실거래가': real_prices,
    '전세가': jeonse_prices,
    '전세가율': jeonse_ratio,
    '건축물_유형': building_types,
    '건축물_용도': building_uses,
    '용도_위반_여부': is_use_violation,
    '불법_건축_여부': is_illegal_build,
    '근저당권_개수': num_mortgages,
    '채권최고액': max_claim_amount,
    '근저당권_설정일_최근': recent_mortgage_date,
    '소유자변경_횟수': num_owner_changes,
    '등기부_갑구_횟수': num_gapgu,
    '등기부_을구_횟수': num_eulgu,
    '신탁_등기여부': is_trust_registered,
    '압류_가압류_등기여부': is_seizure_registered,
    '선순위_채권_존재여부': has_senior_debt,
    '전입_가능여부': is_movein_possible,
    '우선변제권_여부': has_priority_right,
})


# --- 4. '위험여부' 타겟 변수 생성 (모델 학습을 위한 라벨) ---
# 가상의 위험 규칙: 전세가율이 높거나, 압류/신탁이 있거나, 소유자 변경 횟수가 많거나, 근저당권 총액이 높을 때 위험으로 분류
df['총부채비율'] = (df['전세가'] + df['채권최고액']) / df['실거래가']
df['위험여부'] = np.where(
    (df['전세가율'] > 90) | (df['총부채비율'] > 1.2) | (df['압류_가압류_등기여부'] == 1) | (df['신탁_등기여부'] == 1) | (df['소유자변경_횟수'] >= 3),
    1,
    0
)

# --- 5. 최종 데이터프레임 확인 및 CSV 파일로 저장 ---
# '전세가'와 '총부채비율'은 최종 파일에서 제외 (전세가율과 채권최고액으로 충분)
df = df.drop(columns=['전세가', '총부채비율'])

print("샘플 데이터 생성 완료!")
print("데이터셋 미리보기:")
display(df.head())

# CSV 파일로 저장
df.to_csv('real_estate_samples.csv', index=False, encoding='utf-8-sig')
print("\\n'real_estate_samples.csv' 파일이 성공적으로 생성되었습니다.")

샘플 데이터 생성 완료!
데이터셋 미리보기:


,실거래가,전세가율,건축물_유형,건축물_용도,용도_위반_여부,불법_건축_여부,근저당권_개수,채권최고액,근저당권_설정일_최근,소유자변경_횟수,등기부_갑구_횟수,등기부_을구_횟수,신탁_등기여부,압류_가압류_등기여부,선순위_채권_존재여부,전입_가능여부,우선변제권_여부,위험여부
0,44967,72.977962,빌라,주거용,0,0,0,0,2024-11-07,3,5,0,1,0,0,0,0,1
1,38617,74.946267,빌라,주거용,0,0,1,1142,2024-05-09,1,1,1,0,0,1,1,0,0
2,46476,53.610466,아파트,주거용,0,0,0,0,2024-02-19,1,1,1,0,0,0,1,1,0
3,55230,84.220532,다가구주택,주거용,0,0,2,8502,2024-06-13,1,3,2,0,0,1,1,0,0
4,37658,75.160656,빌라,근린생활시설,0,0,0,0,2024-10-04,1,1,0,0,0,0,1,1,0


\n'real_estate_samples.csv' 파일이 성공적으로 생성되었습니다.
